In [3]:
# import module
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import os

# load cifar
transform = transforms.Compose(
    [
    # avoid overfitting
    transforms.RandomHorizontalFlip(),
    transforms.RandomGrayscale(),
    # numpy to tensor
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform1 = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load data: trainset
trainset = torchvision.datasets.CIFAR10(root='./dataset', train=True,
                                        download=False, transform=transform)
# might delete num_workers
# trainset: 100 pics per run/ shuffle / 2 threads
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

# load data: testset
testset = torchvision.datasets.CIFAR10(root='./dataset', train=False,
                                       download=False, transform=transform1)
# trainset: 50 pics per run/ shuffle / 2 threads
testloader = torch.utils.data.DataLoader(testset, batch_size=50,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# define net
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3,64,3,padding=1)
        self.conv2 = nn.Conv2d(64,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        self.conv3 = nn.Conv2d(64,128,3,padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()

        self.conv5 = nn.Conv2d(128,128, 3,padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3,padding=1)
        self.conv7 = nn.Conv2d(128, 128, 1,padding=1)
        self.pool3 = nn.MaxPool2d(2, 2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()

        self.conv8 = nn.Conv2d(128, 256, 3,padding=1)
        self.conv9 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv10 = nn.Conv2d(256, 256, 1, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()

        self.conv11 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv13 = nn.Conv2d(512, 512, 1, padding=1)
        self.pool5 = nn.MaxPool2d(2, 2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()

        self.fc14 = nn.Linear(512*4*4,1024)
        self.drop1 = nn.Dropout2d()
        self.fc15 = nn.Linear(1024,1024)
        self.drop2 = nn.Dropout2d()
        self.fc16 = nn.Linear(1024,10)


    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.bn1(x)
        x = self.relu1(x)


        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.pool4(x)
        x = self.bn4(x)
        x = self.relu4(x)

        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.pool5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        # print(" x shape ",x.size())
        x = x.view(-1,512*4*4)
        x = F.relu(self.fc14(x))
        x = self.drop1(x)
        x = F.relu(self.fc15(x))
        x = self.drop2(x)
        x = self.fc16(x)

        return x
    
    def train_sgd(self,device):
        optimizer = optim.Adam(self.parameters(), lr=0.0001)

        path = 'weights.tar'
        initepoch = 0

        if os.path.exists(path) is False:
            loss = nn.CrossEntropyLoss()
            # optimizer = optim.SGD(self.parameters(),lr=0.01)

        else:
            checkpoint = torch.load(path)
            self.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            initepoch = checkpoint['epoch']
            loss = checkpoint['loss']




        for epoch in range(initepoch,100):  # loop over the dataset multiple times
            timestart = time.time()

            running_loss = 0.0
            total = 0
            correct = 0
            for i, data in enumerate(trainloader, 0):
                # get the inputs
                inputs, labels = data
                inputs, labels = inputs.to(device),labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self(inputs)
                l = loss(outputs, labels)
                l.backward()
                optimizer.step()

                # print statistics
                running_loss += l.item()
                # print("i ",i)
                if i % 500 == 499:  # print every 500 mini-batches
                    print('[%d, %5d] loss: %.4f' %
                          (epoch, i, running_loss / 500))
                    running_loss = 0.0
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    print('Accuracy of the network on the %d tran images: %.3f %%' % (total,
                            100.0 * correct / total))
                    total = 0
                    correct = 0
                    torch.save({'epoch':epoch,
                                'model_state_dict':net.state_dict(),
                                'optimizer_state_dict':optimizer.state_dict(),
                                'loss':loss
                                },path)

            print('epoch %d cost %3f sec' %(epoch,time.time()-timestart))

        print('Finished Training')

    def test(self,device):
        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = self(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy on the testset: %.3f %%' % (
                100.0 * correct / total))

print('test')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net()
net = net.to(device)
net.train_sgd(device)
net.test(device)

test
[0,   499] loss: 1.4162
Accuracy of the network on the 100 tran images: 63.000 %
epoch 0 cost 82.563605 sec
[1,   499] loss: 0.9175
Accuracy of the network on the 100 tran images: 69.000 %
epoch 1 cost 78.481295 sec
[2,   499] loss: 0.7429
Accuracy of the network on the 100 tran images: 70.000 %
epoch 2 cost 78.019937 sec
[3,   499] loss: 0.6353
Accuracy of the network on the 100 tran images: 80.000 %
epoch 3 cost 77.664308 sec
[4,   499] loss: 0.5575
Accuracy of the network on the 100 tran images: 83.000 %
epoch 4 cost 77.935659 sec
[5,   499] loss: 0.5084
Accuracy of the network on the 100 tran images: 81.000 %
epoch 5 cost 77.775403 sec
[6,   499] loss: 0.4549
Accuracy of the network on the 100 tran images: 85.000 %
epoch 6 cost 77.675407 sec
[7,   499] loss: 0.4178
Accuracy of the network on the 100 tran images: 92.000 %
epoch 7 cost 78.121880 sec
[8,   499] loss: 0.3810
Accuracy of the network on the 100 tran images: 86.000 %
epoch 8 cost 78.561928 sec
[9,   499] loss: 0.3528

epoch 74 cost 78.076333 sec
[75,   499] loss: 0.0487
Accuracy of the network on the 100 tran images: 97.000 %
epoch 75 cost 77.938069 sec
[76,   499] loss: 0.0486
Accuracy of the network on the 100 tran images: 100.000 %
epoch 76 cost 77.826086 sec
[77,   499] loss: 0.0470
Accuracy of the network on the 100 tran images: 98.000 %
epoch 77 cost 77.834057 sec
[78,   499] loss: 0.0479
Accuracy of the network on the 100 tran images: 97.000 %
epoch 78 cost 77.462682 sec
[79,   499] loss: 0.0437
Accuracy of the network on the 100 tran images: 97.000 %
epoch 79 cost 77.958624 sec
[80,   499] loss: 0.0429
Accuracy of the network on the 100 tran images: 100.000 %
epoch 80 cost 78.459290 sec
[81,   499] loss: 0.0436
Accuracy of the network on the 100 tran images: 98.000 %
epoch 81 cost 78.214953 sec
[82,   499] loss: 0.0431
Accuracy of the network on the 100 tran images: 97.000 %
epoch 82 cost 77.935410 sec
[83,   499] loss: 0.0428
Accuracy of the network on the 100 tran images: 98.000 %
epoch 83